# Step 3 - Climate Analysis and Exploration

You are now ready to use Python and SQLAlchemy to do basic climate analysis and data exploration on your new weather station tables. All of the following analysis should be completed using SQLAlchemy ORM queries, Pandas, and Matplotlib.

- Create a Jupyter Notebook file called climate_analysis.ipynb and use it to complete your climate analysis and data exporation.

- Choose a start date and end date for your trip. Make sure that your vacation range is approximately 3-15 days total.

- Use SQLAlchemy create_engine to connect to your sqlite database.

- Use SQLAlchemy automap_base() to reflect your tables into classes and save a reference to those classes called Station and Measurement.

In [1]:
# Import Dependencies

# Import Sqlalchemy
import sqlalchemy
# Import the column types necessary that is being used in the database
from sqlalchemy import Column, Integer, String, Float

# Import automap base
from sqlalchemy.ext.automap import automap_base
# Import Session
from sqlalchemy.orm import Session
# Imports the method used for connecting and inspecting DB
from sqlalchemy import create_engine,inspect

# Import Pandas
import pandas as pd
# Import numpy
import numpy as np

# Import matplotlib,pyplot and the seaborn style in matplotlib
import matplotlib
from matplotlib import style
style.use('seaborn')
import matplotlib.pyplot as plt

In [2]:
# Create the hawaii sqlite engine created in database_engineering.ipynb file
engine=create_engine("sqlite:///..\Resources\hawaii.sqlite",echo=False)
# Declare the base as automap_base
Base = automap_base()
# Use the Base class to reflect the database tables
Base.prepare(engine, reflect=True)

In [3]:
# View all the classes mapped to the Base
Base.classes.keys()

['Measurements', 'Stations']

In [4]:
# Save references to the table
Stations = Base.classes.Stations
Measurements = Base.classes.Measurements

## Precipitation Analysis

- Design a query to retrieve the last 12 months of precipitation data.
- Select only the date and prcp values.
- Load the query results into a Pandas DataFrame and set the index to the date column.
- Plot the results using the DataFrame plot method.

In [5]:
# Begin Session to query
session = Session(engine)
import datetime

In [ ]:
# Retrieve the start and end dates
# Query to retrieve the last date in Measurement DB
last_date = session.query(Measurements.Date).order_by(Measurements.Date.desc()).first()
# Converting end date into datetime format
end_date = datetime.datetime.strptime(last_date[0], '%Y-%m-%d')

# Calculating the first date based on the end date
first_date="%d-%d-%d"%(end_date.year-1,end_date.month,end_date.day)
# Converting the start date into datetime format
start_date=datetime.datetime.strptime(first_date, '%Y-%m-%d').strftime("%Y-%m-%d")

# Converting the end date into neccessary datetime format
end_date=end_date.strftime("%Y-%m-%d")

print("Start Date:%s\nEnd Date:%s"%(start_date,end_date))

In [ ]:
# Select only the date and prcp values for the last year
results=session.query(Measurements.Date,Measurements.Prcp).\
        filter(Measurements.Date>=start_date, Measurements.Date<=end_date).\
        order_by(Measurements.Date.desc()).all()

# Converting the results into a dataframe
Precipitation_DF=pd.DataFrame(results)

# Setting the Date column as Index
Precipitation_DF.set_index("Date",inplace=True)

In [ ]:
# View the precipitation table
Precipitation_DF.head(10)

In [ ]:
# Plot the results using DataFrame.plot
Precipitation_DF.plot(rot=45,title="Precipitation from %s to %s"%(start_date,end_date),figsize=(8,5),grid=None,colormap="PuOr_r")
plt.show()

In [ ]:
# Summary statistics of Precipitation Date
Precipitation_DF.describe()

## Station Analysis

- Design a query to calculate the total number of stations.
- Design a query to find the most active stations.
    - List the stations and observation counts in descending order
    - Which station has the highest number of observations?
- Design a query to retrieve the last 12 months of temperature observation data (tobs).
    - Filter by the station with the highest number of observations.
    - Plot the results as a histogram with bins=12.

In [6]:
# Import func from sqlalchemy to count
from sqlalchemy import func

In [ ]:
# Query to find the count of stations 
results=session.query(func.count(Stations.Index)).all()
station_count=results[0][0]
print("Count of Stations: %d"%station_count)

In [ ]:
session.execute("SELECT * FROM Stations").fetchall()

In [ ]:
results=session.query(Measurements.Station,func.count(Measurements.Tobs)).group_by(Measurements.Station).\
                      order_by(func.count(Measurements.Tobs).desc()).all()


In [ ]:
# List the stations and observation counts in descending order
print("Rank\tStation\t\tObservations")
i=1
for result in results:
    print("%d\t%s\t%d"%(i,result[0],result[1]))
    i=i+1

In [ ]:
# Station with highest number of observations
print("Station with Highest Number of Observation\n\n\tStation:\t\t%s\n\tNumber of Observation:\t%d"\
      %(results[0][0],results[0][1]))
# Storing the Station with the most number of observations
station_mostobs=results[0][0]

In [ ]:
# Query to retrieve the last 12 months of Temperature Data
# Use the start date and end date calculated before.
results=session.query(Measurements.Station,Measurements.Date,Measurements.Tobs).\
        filter(Measurements.Date>=start_date, Measurements.Date<=end_date, Measurements.Station == station_mostobs).\
        order_by(Measurements.Date.desc()).all()

In [ ]:
# Converting the results into a dataframe
Temperature_DF=pd.DataFrame(results)

# View the Temperature DataFrame
Temperature_DF.head()

In [ ]:
# Plot the histogram
Temperature_DF.plot.hist(by="Tobs",bins=12,title="Observed Temperature from %s to %s"%(start_date,end_date))
plt.xlabel("Temperature")
plt.ylabel("Frequency of Temperature")
plt.show()

## Temperature Analysis

- Write a function called calc_temps that will accept a start date and end date in the format %Y-%m-%d and return the minimum, average, and maximum temperatures for that range of dates.
- Use the calc_temps function to calculate the min, avg, and max temperatures for your trip using the matching dates from the previous year (i.e. use "2017-01-01" if your trip start date was "2018-01-01")
- Plot the min, avg, and max temperature from your previous query as a bar chart.
    - Use the average temperature as the bar height.
    - Use the peak-to-peak (tmax-tmin) value as the y error bar (yerr).

In [ ]:
start_date = input("Enter your trip start date(YYYY-MM-DD): ")
end_date = input("Enter your trip end date(YYYY-MM-DD): ")

def calc_temps(start_date,end_date):
    results=session.query(func.min(Measurements.Tobs),func.avg(Measurements.Tobs),func.max(Measurements.Tobs)).\
        filter(Measurements.Date>=start_date, Measurements.Date<=end_date).all()
    Tmin = results[0][0]
    Tavg = results[0][1]
    Tmax = results[0][2]
    print("\nTemperature Summary from %s to %s"%(start_date,end_date))
    print("--------------------------------------------------")
    print("  Minimum Temperature: %d"%Tmin)
    print("  Average Temperature: %d"%Tavg)
    print("  Maximum Temperature: %d"%Tmax)
    return Tmin, Tavg, Tmax
temperature_results=calc_temps(start_date,end_date)

In [ ]:
# Plot bar graph
plt.figure(figsize=(2,10))
error= temperature_results[2]-temperature_results[0]
plt.bar(0,temperature_results[1],yerr=error,color="coral")
plt.ylim(0,102)
plt.xticks([0])
plt.ylabel("Temp (°F)")
plt.title("Trip Avg Temp")
plt.grid(b=True,which="minor",axis="y")
plt.show()

## Optional Recommended Analysis

### Analysis 1
- The following are optional challenge queries. These are highly recommended to attempt, but not required for the homework.
    - Calculate the rainfall per weather station using the previous year's matching dates.

In [37]:
# Rainfall per weather station using the previous year's matching dates

# Input the start date and end date
start_date = input("Enter your trip start date(YYYY-MM-DD): ")
end_date = input("Enter your trip end date(YYYY-MM-DD): ")

# Convert date into datetime object
start_date = datetime.datetime.strptime(start_date, "%Y-%m-%d")
end_date = datetime.datetime.strptime(end_date, "%Y-%m-%d")

# Convert the start date and end date to last years dates
ly_start_date = (start_date-datetime.timedelta(365)).strftime("%Y-%m-%d")
ly_end_date = (end_date-datetime.timedelta(365)).strftime("%Y-%m-%d")


# Query to find the rainfall for each statition
query = session.query(Measurements.Station, func.sum(Measurements.Prcp)).\
          filter(Measurements.Date >= ly_start_date, Measurements.Date <= ly_end_date).\
          group_by(Measurements.Station).\
          order_by(Measurements.Station)

# Store the results in a dataframe
rainfall_per_station = pd.read_sql(query.statement, query.session.bind)
rainfall_per_station.set_index("Station",inplace=True)
rainfall_per_station = rainfall_per_station.rename(columns={"sum_1":"Rainfall"})
# View the result
rainfall_per_station


Enter your trip start date(YYYY-MM-DD): 2018-06-08
Enter your trip end date(YYYY-MM-DD): 2018-06-15


,Rainfall
Station,
USC00513117,1.480000
USC00514830,1.640000
USC00516128,4.820644
USC00517948,0.803218
USC00519281,2.630000
USC00519397,0.140000
USC00519523,0.610000


### Analysis 2
- Calculate the daily normals. Normals are the averages for min, avg, and max temperatures.
    - Create a function called daily_normals that will calculate the daily normals for a specific date. This date string will be in the format %m-%d. Be sure to use all historic tobs that match that date string.
    - Create a list of dates for your trip in the format %m-%d. Use the daily_normals function to calculate the normals for each date string and append the results to a list.
    - Load the list of daily normals into a Pandas DataFrame and set the index equal to the date.
    - Use Pandas to plot an area plot (stacked=False) for the daily normals.

In [ ]:
# List to hold the dates of the trip
date_list=["01-01","01-02","01-03","01-04","01-05","01-06","01-07"]
# Intialize a list to store the results of the list
daily_results=[]

In [ ]:
# Function to calculate the min,avg and max for a date and return those values
def daily_normals(daily_date):
    results= session.query(func.min(Measurements.Tobs),func.avg(Measurements.Tobs),func.max(Measurements.Tobs))\
            .filter(Measurements.Date.like("%-"+daily_date)).all()
    return results

# For each date in the date_list calculate the min,avg and max and store to a result list that also has the date
for date in date_list:
    date_result=daily_normals(date)
    daily_results.append(["2018-"+date,date_result[0][0],date_result[0][1],date_result[0][2]])

In [ ]:
# Convert the list into a DaataFrame
Daily_Normals_DF=pd.DataFrame(daily_results)
# Rename the columns of the DataFrame
Daily_Normals_DF = Daily_Normals_DF.rename(columns={0:"Date", 1:"Tmin", 2:"Tavg", 3:"Tmax"})
# Set the index as Date for the Dataframe
Daily_Normals_DF.set_index("Date",inplace=True)

# View the dataframe
Daily_Normals_DF.head()

In [ ]:
Daily_Normals_DF.plot(kind='area', stacked=False, alpha=0.5,  colormap='Set1', rot=45,\
                     title= "Daily Normal Temperature(°F)")
plt.ylabel("Temp (°F)")
plt.show()